# Load goodreads data
Og lidt oprydning.

In [ ]:
import main
import pandas as pd

# load goodreads data, clean whitespace from author names and filter out unread books
df = main.read_data()
df['Author'] = df['Author'].str.replace(r'\s+', ' ', regex=True)
df['Author'].unique()
df=df[df['Read Count'] > 0]

# collapse the list on author and add book count and average 'Average Rating'
df = df.groupby('Author').apply(lambda x: pd.Series({
    'Title': ', '.join(x['Title']),
    'Average Rating': x['Average Rating'].mean(),
})).reset_index()

df['Books'] = df['Title'].str.split(', ')
df.drop('Title', axis=1, inplace=True)

# Author country lookup


## Funktion til at kalde wiki datas SPARQL endpoint


In [ ]:
import sys
import re
from SPARQLWrapper import SPARQLWrapper, JSON

def execute_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s (christian@dalager.com)" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


def get_author_countries(author_name):
    author_name = re.sub(r"\s+", " ", author_name)
    author_name = author_name.strip()

    endpoint_url = "https://query.wikidata.org/sparql"

    query = (
        """SELECT ?countryLabel WHERE {
        ?author ?label "%s"@en.
        ?author wdt:P27 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }"""
        % author_name
    )

    results = execute_query(endpoint_url, query)

    bindings = results["results"]["bindings"]
    countries = list({entry["countryLabel"]["value"] for entry in bindings})

    return countries


## Kald data apiet
Det tager 1-2 minutter at lave 369 opslag

In [ ]:
# extract all author names from "Author" to a unique list
import pandas as pd
authors = df['Author'].unique()

print(f'Requesting {len(authors)} authors')

# Create a list to collect author and their countries
author_country_list = []

# Iterate over all authors and get the countries  (1.5 min)
for author in authors:
    countries = get_author_countries(author)  # Make sure this returns a list of countries
    author_country_list.append({'Author': author, 'Countries': countries})

df_authorcountries = pd.DataFrame(author_country_list)
    
# save to csv
df_authorcountries.to_csv('../data/author_countries.csv', index=False)


# Begynd at arbejde med listerne

In [ ]:
df_authorcountries = pd.read_csv('../data/author_countries.csv')

import ast
for index, row in df_authorcountries.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_authorcountries.at[index, 'Countries'] = countries

# Manually located authors' countries in the dataset

In [ ]:
df_manually_placed = pd.read_csv('../data/manually_placed.csv')

for index, row in df_manually_placed.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_manually_placed.at[index, 'Countries'] = countries

df_manually_placed.rename(columns={'Countries': 'Countries_Manually_Placed'}, inplace=True)
merged_df = pd.merge(df_authorcountries, df_manually_placed[['Author', 'Countries_Manually_Placed']], on='Author', how='left', validate='1:1')
# Replace the 'Countries' column with the corrected countries where available
merged_df['Countries'] = merged_df['Countries_Manually_Placed'].fillna(merged_df['Countries'])

# Drop the 'Countries_Manually_Placed' column if you don't need it anymore
merged_df=merged_df.drop(columns=['Countries_Manually_Placed'])

df_authorcountries = merged_df


In [263]:
df_authorcountries





,Author,Countries
0,A.G. Riddle,[United States of America]
1,Adam Lashinsky,[United States of America]
2,Adrian Tchaikovsky,[United Kingdom]
3,Agatha Christie,"[United Kingdom of Great Britain and Ireland, United Kingdom]"
4,Alan Moore,"[Republic of Ireland, Australia, United Kingdom, United Kingdom of Great Britain and Ireland, Russia]"
5,Albert Camus,[France]
6,Alberto Brandolini,[Italy]
7,Aldous Huxley,[United Kingdom]
8,Alexandre Dumas,"[France, Canada, Haiti]"
9,Amina Elmi,"[Somalia, Denmark]"


# Complete the final dataset

In [ ]:
df_final = pd.merge(df, df_authorcountries, on='Author', how='left', validate='1:1')
df_final.to_csv('../data/myauthorlist_with_countries_and_books.csv', index=False)

In [267]:
country_mapping = {
    'Ancient Rome': 'Italy',
    'Austria-Hungary': 'Austria',
    'Cisleithania': 'Austria',
    'Classical Athens': 'Greece',
    'Czechoslovakia': 'Czech Republic',
    'Dutch Republic': 'Netherlands',
    'England': 'United Kingdom',
    'German Empire': 'Germany',
    'Kingdom of Denmark': 'Denmark',
    'Kingdom of England': 'United Kingdom',
    'Kingdom of Great Britain': 'United Kingdom',
    'Kingdom of Italy': 'Italy',
    'Kingdom of the Netherlands': 'Netherlands',
    'Nazi Germany': 'Germany',
    'Russian Empire': 'Russia',
    'Russian Socialist Federative Soviet Republic': 'Russia',
    'Russian Soviet Federative Socialist Republic': 'Russia',
    'Scotland': 'United Kingdom',
    'United Kingdom of Great Britain and Ireland': 'United Kingdom',
    'Ireland':'Republic of Ireland',
    'Weimar Republic': 'Germany',
    'statelessness': None
    # Add more mappings as needed
}

# Function to clean country names
def clean_country_list(country_list):
    replaced= [country_mapping.get(country, country) for country in country_list if country_mapping.get(country, country) is not None]
    return list(set(replaced))

# Apply the function to the 'Countries' column
df_final['Countries'] = df_final['Countries'].apply(clean_country_list)



In [268]:
df_final

,Author,Average Rating,Books,Countries
0,A.G. Riddle,3.876667,"[Departure, The Atlantis Plague (The Origin Mystery, #2), The Atlantis Gene (The Origin Mystery, #1)]",[United States of America]
1,Adam Lashinsky,3.830000,[Inside Apple -- From Steve Jobs Down to the Janitor: How America's Most Successful—and Most Secretive—Big Company Really Works],[United States of America]
2,Adrian Tchaikovsky,4.300000,"[Children of Time (Children of Time, #1)]",[United Kingdom]
3,Agatha Christie,4.280000,[And Then There Were None],[United Kingdom]
4,Alan Moore,4.380000,[Watchmen],"[Australia, Republic of Ireland, Russia, United Kingdom]"
5,Albert Camus,4.020000,"[The Plague, The Stranger]",[France]
6,Alberto Brandolini,3.850000,[Introducing EventStorming],[Italy]
7,Aldous Huxley,3.990000,[Brave New World],[United Kingdom]
8,Alexandre Dumas,4.300000,[The Count of Monte Cristo],"[France, Canada, Haiti]"
9,Amina Elmi,4.130000,[Barbar: [Tavshedens objekt]],"[Somalia, Denmark]"


# Show me the countries!

In [265]:
# list of uniqe countries
cs=list(df_final['Countries'])
cs = {item for sublist in cs for item in sublist}
cs = list(cs)
cs.sort()
cs




['Australia',
 'Austria',
 'Brazil',
 'Canada',
 'Chile',
 'Colombia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Greenland',
 'Haiti',
 'Hungary',
 'India',
 'Iran',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Lebanon',
 'Malaysia',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Northern Ireland',
 'Norway',
 "People's Republic of China",
 'Poland',
 'Republic of Ireland',
 'Russia',
 'Senegal',
 'Somalia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'United States of America',
 'Vietnam',
 'Wales']